In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# using tensorflow>=2.2.1 with built-in Keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, PReLU
import tensorflow.keras.utils as np_utils

In [ ]:
TEST_DATA_ROOT = '~/workspace/datarobot-user-models/tests/testdata'
BINARY_DATA = os.path.join(
    TEST_DATA_ROOT, 'iris_binary_training.csv'
)
REGRESSION_DATA = os.path.join(
    TEST_DATA_ROOT, 'juniors_3_year_stats_regression.csv'
)
MULTICLASS_DATA = os.path.join(
    TEST_DATA_ROOT, 'skyserver_sql2_27_2018_6_51_39_pm.csv'
)

bin_X = pd.read_csv(BINARY_DATA)
bin_y = bin_X.pop('Species')

reg_X = pd.read_csv(REGRESSION_DATA).fillna(0)
reg_y = reg_X.pop('Grade 2014')

multi_X = pd.read_csv(MULTICLASS_DATA)
multi_y = multi_X.pop('class')

In [ ]:
bin_model = Sequential([
    Dense(50, input_dim=bin_X.shape[1]),
    Activation('relu'),
    Dropout(0.2),
    Dense(100),
    PReLU(),
    Dense(1),
    Activation('sigmoid')
])
bin_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
bin_target_encoder = LabelEncoder()
bin_target_encoder.fit(bin_y)
bin_model.fit(bin_X, bin_target_encoder.transform(bin_y), epochs=10, batch_size=32)

reg_model = Sequential([
    Dense(50, input_dim=reg_X.shape[1]),
    Activation('relu'),
    Dropout(0.2),
    Dense(100),
    PReLU(),
    Dense(1),
])
reg_model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
reg_model.fit(reg_X, reg_y, epochs=10, batch_size=32)

multi_target_encoder = LabelEncoder()
multi_target_encoder.fit(multi_y)
multi_model = Sequential()
multi_model.add(Dense(8, input_dim=multi_X.shape[1], activation='relu'))
multi_model.add(Dense(len(multi_target_encoder.classes_), activation='softmax'))
multi_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
multi_model.fit(multi_X, np_utils.to_categorical(multi_target_encoder.transform(multi_y)), epochs=10, batch_size=32)

In [ ]:
print(bin_model.predict(pd.DataFrame(bin_X)))
print(reg_model.predict(pd.DataFrame(reg_X)))
print(multi_model.predict(pd.DataFrame(multi_X)))

In [ ]:
FIXTURE_ROOT = '~/workspace/datarobot-user-models/tests/fixtures/drop_in_model_artifacts'
bin_model.save(os.path.expanduser(os.path.join(FIXTURE_ROOT, 'keras_bin.h5')))
reg_model.save(os.path.expanduser(os.path.join(FIXTURE_ROOT, 'keras_reg.h5')))
multi_model.save(os.path.expanduser(os.path.join(FIXTURE_ROOT, 'keras_multi.h5')))